In [1]:
!pip install pyro-ppl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 10.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.7 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDelta

In [3]:
# Carica i dati
X_array = np.load("/kaggle/input/bow-xarray/BoW_X_Array.npz")["arr_0"]
X_tensor = torch.tensor(X_array, dtype=torch.float)
print("XArray added")

# Opzionale: usa GPU se disponibile
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

X_tensor = X_tensor.to(device)

num_docs, vocab_size = X_tensor.shape
K_values = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
results = []

for K in K_values:
    print(f"\n--- Training LDA with K = {K} ---")

    def lda_model(data):
        with pyro.plate("topics", K):
            topic_words = pyro.sample("topic_words", dist.Dirichlet(torch.ones(vocab_size).to(device)))
        with pyro.plate("documents", num_docs):
            doc_topics = pyro.sample("doc_topics", dist.Dirichlet(torch.ones(K).to(device)))
            word_dists = torch.matmul(doc_topics, topic_words)
            logits = torch.matmul(doc_topics, topic_words).log()
            pyro.sample("doc_words", dist.Multinomial(total_count=100, logits=logits), obs=data)

    pyro.clear_param_store()
    guide = AutoDelta(lda_model)
    svi = SVI(lda_model, guide, pyro.optim.Adam({"lr": 0.01}), loss=Trace_ELBO())

    for step in range(500):
        loss = svi.step(X_tensor)

    posterior = guide()
    doc_topics = posterior["doc_topics"]
    topic_usage = doc_topics.sum(dim=0).detach().cpu().numpy()

    # Statistiche extra
    loss_per_doc = loss / num_docs
    entropy = -(doc_topics * doc_topics.log()).sum(dim=1).mean().item()
    avg_active_per_doc = (doc_topics > 0.05).sum(dim=1).float().mean().item()
    num_active_topics = (topic_usage > 5.0).sum()

    results.append({
        "K": K,
        "Final Loss": float(loss),
        "Loss per Doc": float(loss_per_doc),
        "Entropy": float(entropy),
        "Avg Active Topics/Doc": float(avg_active_per_doc),
        "Active Topics (global)": int(num_active_topics)
    })

results_df = pd.DataFrame(results)
results_df.to_csv("results_k_selection.csv", index=False)


XArray added
cuda

--- Training LDA with K = 5 ---

--- Training LDA with K = 10 ---

--- Training LDA with K = 15 ---

--- Training LDA with K = 20 ---

--- Training LDA with K = 25 ---

--- Training LDA with K = 30 ---

--- Training LDA with K = 35 ---

--- Training LDA with K = 40 ---

--- Training LDA with K = 45 ---

--- Training LDA with K = 50 ---

--- Training LDA with K = 55 ---

--- Training LDA with K = 60 ---

--- Training LDA with K = 65 ---

--- Training LDA with K = 70 ---

--- Training LDA with K = 75 ---

--- Training LDA with K = 80 ---

--- Training LDA with K = 85 ---

--- Training LDA with K = 90 ---


In [4]:
results_df

,K,Final Loss,Loss per Doc,Entropy,Avg Active Topics/Doc,Active Topics (global)
0,5,-4.243077e+04,-7.614998,0.377198,1.628141,5
1,10,-3.917967e+05,-70.315278,0.532416,1.924803,10
2,15,-7.485161e+05,-134.335267,0.613301,2.062096,15
3,20,-1.116392e+06,-200.357555,0.703568,2.164393,20
4,25,-1.486718e+06,-266.819421,0.804096,2.395908,25
5,30,-1.860040e+06,-333.819168,0.820487,2.374013,30
6,35,-2.237448e+06,-401.552029,0.874397,2.456389,35
7,40,-2.618543e+06,-469.946670,0.962098,2.622039,40
8,45,-3.000189e+06,-538.440243,1.102405,3.023690,45
9,50,-3.386729e+06,-607.812133,0.990354,2.613963,50


In [5]:
import numpy as np
import pandas as pd
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
from pyro.infer.autoguide import AutoDelta
from sklearn.model_selection import train_test_split

# === PARAMETRI ===
K = 60
num_steps_train = 1000
num_steps_test_svi = 500
num_samples_mcmc = 800
warmup_mcmc = 300
dirichlet_alpha = 0.1
device = "cuda" if torch.cuda.is_available() else "cpu"


In [6]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer

df_sms = pd.read_csv("/kaggle/input/sms-cleaned/lda_df.csv")

# Convert stringified lists to actual lists
df_sms["tokens"] = df_sms["tokens"].apply(ast.literal_eval)

# Join tokens into space-separated strings
df_sms["joined_tokens"] = df_sms["tokens"].apply(lambda tokens: " ".join(tokens))

# Create BoW matrix
vectorizer = CountVectorizer()
X_array = vectorizer.fit_transform(df_sms["joined_tokens"]).toarray()


In [7]:
import numpy as np
import pandas as pd
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
from pyro.infer.autoguide import AutoDelta
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import ast

# === PARAMETRI ===
K = 60
num_steps_train = 1000
num_steps_test_svi = 500
num_samples_mcmc = 800
warmup_mcmc = 300
dirichlet_alpha = 0.1
device = "cuda" if torch.cuda.is_available() else "cpu"

# === CARICAMENTO E PREPARAZIONE DATI ===
df_sms = pd.read_csv("/kaggle/input/sms-cleaned/lda_df.csv")  # includes 'label', 'message', 'tokens'
df_sms["tokens"] = df_sms["tokens"].apply(ast.literal_eval)
df_sms["joined_tokens"] = df_sms["tokens"].apply(lambda tokens: " ".join(tokens))

vectorizer = CountVectorizer()
X_array = vectorizer.fit_transform(df_sms["joined_tokens"]).toarray()

# === TRAIN/TEST SPLIT ===
X_train_np, X_test_np, y_train, y_test = train_test_split(
    X_array,
    df_sms["label"].values,
    test_size=0.2,
    stratify=df_sms["label"],
    random_state=42
)

# Save labels for classification later
pd.DataFrame({"label": y_train}).to_csv("sms_train.csv", index=False)
pd.DataFrame({"label": y_test}).to_csv("sms_test.csv", index=False)

In [ ]:
# Convert to torch tensors
X_train = torch.tensor(X_train_np, dtype=torch.float).to(device)
X_test = torch.tensor(X_test_np, dtype=torch.float).to(device)
num_train, vocab_size = X_train.shape
num_test = X_test.shape[0]

# === MODELLO LDA PER TRAINING ===
def lda_model(data, num_docs):
    with pyro.plate("topics", K):
        topic_words = pyro.sample("topic_words", dist.Dirichlet(torch.ones(vocab_size).to(device)))
    with pyro.plate("documents", num_docs):
        doc_topics = pyro.sample("doc_topics", dist.Dirichlet(dirichlet_alpha * torch.ones(K).to(device)))
        logits = torch.matmul(doc_topics, topic_words).log()
        pyro.sample("doc_words", dist.Multinomial(total_count=100, logits=logits), obs=data)

# === TRAIN LDA SU TRAIN SET CON SVI ===
pyro.clear_param_store()
guide = AutoDelta(lambda data: lda_model(data, num_train))
svi = SVI(lambda data: lda_model(data, num_train), guide, pyro.optim.Adam({"lr": 0.01}), loss=Trace_ELBO())

print(f"\n🚀 Training LDA on train set with SVI...")
for step in range(num_steps_train):
    loss = svi.step(X_train)
    if step % 100 == 0:
        print(f"[step {step}] loss = {loss:.2f}")

posterior = guide(X_train)
topic_words = posterior["topic_words"].detach()
doc_topics_train = posterior["doc_topics"].detach().cpu().numpy()

pd.DataFrame(doc_topics_train, columns=[f"topic_{i}" for i in range(K)]).to_csv("doc_topics_train_K60.csv", index=False)
print("DS train saved!")

# === INFERENZA SVI SUL TEST SET ===
def lda_inference_svi_model(data, topic_words_fixed, num_docs):
    with pyro.plate("documents", num_docs):
        doc_topics = pyro.sample("doc_topics", dist.Dirichlet(dirichlet_alpha * torch.ones(K).to(device)))
        logits = torch.matmul(doc_topics, topic_words_fixed).log()
        pyro.sample("doc_words", dist.Multinomial(total_count=100, logits=logits), obs=data)

guide_test_svi = AutoDelta(lambda data: lda_inference_svi_model(data, topic_words, num_test))
svi_test = SVI(lambda data: lda_inference_svi_model(data, topic_words, num_test),
               guide_test_svi, pyro.optim.Adam({"lr": 0.01}), loss=Trace_ELBO())

pyro.clear_param_store()
print("\n⚡ Running SVI inference on test set...")
for step in range(num_steps_test_svi):
    svi_test.step(X_test)
    if step % 100 == 0:
        print(f"[step {step}]")

posterior_test_svi = guide_test_svi(X_test)
doc_topics_test_svi = posterior_test_svi["doc_topics"].detach().cpu().numpy()

pd.DataFrame(doc_topics_test_svi, columns=[f"topic_{i}" for i in range(K)]).to_csv("doc_topics_test_K60_svi.csv", index=False)

# === INFERENZA MCMC SUL TEST SET ===
def lda_inference_mcmc_model(data, topic_words_fixed, num_docs):
    with pyro.plate("documents", num_docs):
        doc_topics = pyro.sample("doc_topics", dist.Dirichlet(dirichlet_alpha * torch.ones(K).to(device)))
        logits = torch.matmul(doc_topics, topic_words_fixed).log()
        pyro.sample("doc_words", dist.Multinomial(total_count=100, logits=logits), obs=data)

mcmc_model = lambda data: lda_inference_mcmc_model(data, topic_words, num_test)

pyro.clear_param_store()
nuts_kernel = NUTS(mcmc_model)
mcmc = MCMC(nuts_kernel, num_samples=num_samples_mcmc, warmup_steps=warmup_mcmc, num_chains=1)

print("\n🧊 Running MCMC inference on test set...")
mcmc.run(X_test)

mcmc_samples = mcmc.get_samples()
doc_topics_test_mcmc = mcmc_samples["doc_topics"].detach().cpu().numpy()

doc_topics_mcmc_mean = doc_topics_test_mcmc.mean(axis=0)
doc_topics_mcmc_std = doc_topics_test_mcmc.std(axis=0)

pd.DataFrame(doc_topics_mcmc_mean, columns=[f"topic_{i}" for i in range(K)]).to_csv("doc_topics_test_K60_mcmc_mean.csv", index=False)
pd.DataFrame(doc_topics_mcmc_std, columns=[f"topic_{i}" for i in range(K)]).to_csv("doc_topics_test_K60_mcmc_std.csv", index=False)

print("\u2705 All results saved successfully.")


🚀 Training LDA on train set with SVI...
[step 0] loss = -3478207.34
[step 100] loss = -3590968.77
[step 200] loss = -3937175.31
[step 300] loss = -4266887.02
[step 400] loss = -4660791.03
[step 500] loss = -5097602.12
[step 600] loss = -5623749.00
[step 700] loss = -6222733.75
[step 800] loss = -6854969.20
[step 900] loss = -7495493.97
DS train saved!

⚡ Running SVI inference on test set...
[step 0]
[step 100]
[step 200]
[step 300]
[step 400]

🧊 Running MCMC inference on test set...


Sample:  75%|███████▍  | 821/1100 [1:23:24,  5.16s/it, step size=9.55e-05, acc. prob=0.787]

Confronto tra SVI e MCMC

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

svi = pd.read_csv("/kaggle/working/doc_topics_test_K60_svi.csv").iloc[0]
mcmc_mean = pd.read_csv("/kaggle/working/doc_topics_test_K60_mcmc_mean.csv").iloc[0]
mcmc_std = pd.read_csv("/kaggle/working/doc_topics_test_K60_mcmc_std.csv").iloc[0]

plt.figure(figsize=(12, 4))
plt.plot(svi, label="SVI")
plt.plot(mcmc_mean, label="MCMC Mean", linestyle="--")
plt.fill_between(range(60), mcmc_mean - mcmc_std, mcmc_mean + mcmc_std, alpha=0.3, label="MCMC ± std")
plt.title("Distribuzione di topic per un documento di test")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Carica i file
doc_svi = pd.read_csv("/kaggle/working/doc_topics_test_K60_svi.csv")
doc_mcmc_mean = pd.read_csv("/kaggle/working/doc_topics_test_K60_mcmc_mean.csv")
doc_mcmc_std = pd.read_csv("/kaggle/working/doc_topics_test_K60_mcmc_std.csv")

# Prendiamo i primi 10 documenti
N = 10
svi_batch = doc_svi.iloc[:N].values
mcmc_mean_batch = doc_mcmc_mean.iloc[:N].values
mcmc_std_batch = doc_mcmc_std.iloc[:N].values

# === METRICHE ===
def entropy(p):
    p = np.clip(p, 1e-12, 1.0)  # evita log(0)
    return -np.sum(p * np.log(p), axis=1)

def active_topics(p, threshold=0.05):
    return (p > threshold).sum(axis=1)

# Calcoli
svi_entropy = entropy(svi_batch)
mcmc_entropy = entropy(mcmc_mean_batch)

svi_active = active_topics(svi_batch)
mcmc_active = active_topics(mcmc_mean_batch)

mcmc_uncertainty = mcmc_std_batch.mean(axis=1)

# === RISULTATI AGGREGATI ===
print("📊 METRICHE MEDIE SU 10 DOCUMENTI:\n")
print(f"Entropia SVI         : {svi_entropy.mean():.4f}")
print(f"Entropia MCMC        : {mcmc_entropy.mean():.4f}")
print(f"Topic attivi SVI     : {svi_active.mean():.2f}")
print(f"Topic attivi MCMC    : {mcmc_active.mean():.2f}")
print(f"Incertezza MCMC (std): {mcmc_uncertainty.mean():.4f}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(svi_entropy, label="SVI", marker='o')
plt.plot(mcmc_entropy, label="MCMC", marker='x')
plt.title("Entropia per documento (SVI vs MCMC)")
plt.xlabel("Documento")
plt.ylabel("Entropia")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(svi_active, label="SVI", marker='o')
plt.plot(mcmc_active, label="MCMC", marker='x')
plt.title("Topic attivi (>0.05) per documento")
plt.xlabel("Documento")
plt.ylabel("# topic attivi")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.boxplot(mcmc_std_batch.mean(axis=1), vert=False)
plt.title("Distribuzione dell’incertezza media per documento (MCMC)")
plt.xlabel("Varianza media dei topic")
plt.tight_layout()
plt.show()


# Start with Bayesian Classifier

In [22]:
df_k60 = pd.read_csv("/kaggle/working/doc_topics_train_K60.csv")
df_sms = pd.read_csv(
    "/kaggle/input/smsdataset/SMSSpamCollection", sep="\t", header=None, names=["label", "message"]
)
# df_sms.head()
df_sms["label"] = df_sms["label"].map({"ham": 0, "spam": 1})
df_k60["label"] = df_sms["label"]
bow_array = np.load("/kaggle/input/bow-xarray/BoW_X_Array.npz")["arr_0"]
df_k60.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_51,topic_52,topic_53,topic_54,topic_55,topic_56,topic_57,topic_58,topic_59,label
0,1.000000e+00,5.490356e-12,5.500995e-12,5.935029e-12,5.934962e-12,5.500596e-12,5.501006e-12,5.934859e-12,5.935007e-12,5.500953e-12,...,5.935007e-12,5.934996e-12,5.500953e-12,5.500963e-12,5.935007e-12,5.934996e-12,5.500953e-12,5.500963e-12,5.935007e-12,0
1,1.120545e-13,1.262004e-13,1.266233e-13,6.726096e-09,1.926159e-08,1.266194e-13,1.266245e-13,9.999993e-01,2.872921e-08,1.266240e-13,...,2.901031e-08,2.895216e-08,1.266242e-13,1.266242e-13,2.901031e-08,2.895221e-08,1.266242e-13,1.266242e-13,2.901031e-08,0
2,3.464618e-13,2.145838e-08,1.414600e-07,4.427267e-13,4.429049e-13,5.805587e-05,2.683781e-07,4.431238e-13,4.431956e-13,4.698514e-05,...,4.431956e-13,4.431956e-13,5.818885e-03,6.797621e-03,4.431956e-13,4.431956e-13,6.871266e-03,6.662033e-03,4.431956e-13,1
3,9.117645e-14,1.053106e-13,1.053608e-13,3.049746e-09,2.115192e-08,1.053258e-13,1.053269e-13,4.779859e-08,1.177722e-06,1.053773e-13,...,8.680254e-06,4.983503e-05,1.053773e-13,1.053779e-13,8.680262e-06,5.056385e-05,1.053773e-13,1.053779e-13,8.680246e-06,0
4,6.902157e-14,8.031758e-14,8.042826e-14,2.778378e-09,2.164246e-08,8.034501e-14,8.034746e-14,1.424361e-08,2.850565e-07,8.034730e-14,...,1.413670e-05,1.250400e-05,8.034715e-14,8.034715e-14,1.413582e-05,1.276869e-05,8.034715e-14,8.034715e-14,1.413578e-05,0


## BAYESIAN CLASSIFIER

In [31]:
import pandas as pd
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
from pyro.infer.autoguide import AutoDelta
from sklearn.metrics import classification_report, roc_auc_score

# === LOAD DATASETS ===
X_train = pd.read_csv("/kaggle/working/doc_topics_train_K60.csv").values
X_test = pd.read_csv("/kaggle/working/doc_topics_test_K60_mcmc_mean.csv").values

y_train = (pd.read_csv("sms_train.csv")['label'] == 'spam').astype(float).values
y_test = (pd.read_csv("sms_test.csv")['label'] == 'spam').astype(float).values

# Convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)

# === MODEL ===
def model(X, y=None):
    num_features = X.shape[1]
    beta = pyro.sample("beta", dist.Normal(torch.zeros(num_features), torch.ones(num_features)).to_event(1))
    bias = pyro.sample("bias", dist.Normal(0., 1.))
    with pyro.plate("data", X.shape[0]):
        logits = (X @ beta) + bias
        pyro.sample("obs", dist.Bernoulli(logits=logits), obs=y)


# === SVI INFERENCE ===
pyro.clear_param_store()
guide = AutoDelta(model)
svi = SVI(model, guide, pyro.optim.Adam({"lr": 0.01}), loss=Trace_ELBO())

for step in range(1000):
    loss = svi.step(X_train, y_train)
    if step % 100 == 0:
        print(f"[SVI step {step}] loss = {loss:.2f}")

posterior = guide(X_train, y_train)
weights = posterior["beta"]
bias = posterior["bias"]

# Predictions
logits_test = (X_test @ weights) + bias
probs_test = torch.sigmoid(logits_test).detach().numpy()
pred_test = (probs_test > 0.5).astype(int)

# Evaluation
print("\n=== Evaluation on test set ===")
print(classification_report(y_test, pred_test, target_names=["ham", "spam"]))
print(f"AUC: {roc_auc_score(y_test, probs_test):.3f}")


[SVI step 0] loss = 2546.42
[SVI step 100] loss = 1829.73
[SVI step 200] loss = 1808.07
[SVI step 300] loss = 1804.87
[SVI step 400] loss = 1803.15
[SVI step 500] loss = 1801.75
[SVI step 600] loss = 1800.54
[SVI step 700] loss = 1799.52
[SVI step 800] loss = 1798.68
[SVI step 900] loss = 1798.02

=== Evaluation on test set ===
              precision    recall  f1-score   support

         ham       0.87      1.00      0.93       966
        spam       0.00      0.00      0.00       149

    accuracy                           0.87      1115
   macro avg       0.43      0.50      0.46      1115
weighted avg       0.75      0.87      0.80      1115

AUC: 0.483


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
